In [1]:
import openai
import os
import wikipedia

# 1.) Set up OpenAI and the enviornment


In [2]:
apikey = "sdk-daLaJ7KKC4ExUyGhY20gfWHkT3ghBlbkFJaXFbx9dKHjaNvGFrbyFChBs"
openai.api_key = apikey

In [3]:
client = openai.OpenAI(
    api_key = openai.api_key
)

In [4]:
dir(wikipedia)

['API_URL',
 'BeautifulSoup',
 'Decimal',
 'DisambiguationError',
 'HTTPTimeoutError',
 'ODD_ERROR_MESSAGE',
 'PageError',
 'RATE_LIMIT',
 'RATE_LIMIT_LAST_CALL',
 'RATE_LIMIT_MIN_WAIT',
 'RedirectError',
 'USER_AGENT',
 'WikipediaException',
 'WikipediaPage',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'cache',
 'datetime',
 'debug',
 'donate',
 'exceptions',
 'geosearch',
 'languages',
 'page',
 'random',
 're',
 'requests',
 'search',
 'set_lang',
 'set_rate_limiting',
 'set_user_agent',
 'stdout_encode',
 'suggest',
 'summary',
 'sys',
 'time',
 'timedelta',
 'unicode_literals',
 'util',
 'wikipedia']

# 2.) Use the wikipedia api to get a function that pulls in the text of a wikipedia page

In [5]:
# To loop through incorrect content in Wikipedia
page_titles = ["Alternative energy", "Oil Price"]
page_title = page_titles[0]

In [6]:
search_results = wikipedia.search(page_title)
search_results

['Renewable energy',
 'Sustainable energy',
 'Alternative fuel vehicle',
 'Alternative Energy Development Board',
 'French Alternative Energies and Atomic Energy Commission',
 'Energy development',
 'Alternative Energy (composition)',
 'Wind power',
 'Alternative Energy Institute',
 'Chevron Corporation']

In [7]:
page = wikipedia.page(search_results[0])
page.content

'Renewable energy, green energy, or low-carbon energy is energy from renewable resources that are naturally replenished on a human timescale. Renewable resources include sunlight, wind, the movement of water, and geothermal heat. Although most renewable energy sources are sustainable, some are not. For example, some biomass sources are considered unsustainable at current rates of exploitation. Renewable energy is often used for electricity generation, heating and cooling. Renewable energy projects are typically large-scale, but they are also suited to rural and remote areas and developing countries, where energy is often crucial in human development.Renewable energy is often deployed together with further electrification, which has several benefits: electricity can move heat or objects efficiently, and is clean at the point of consumption. From 2011 to 2021, renewable energy grew from 20% to 28% of global electricity supply. Use of fossil energy shrank from 68% to 62%, and nuclear from

In [8]:
#page.content
def get_wikipedia_content(page_title):
    search_results = wikipedia.search(page_title)
    page = wikipedia.page(search_results[0])
    return(page.content)

In [9]:
content = get_wikipedia_content(page_title)

# 3.) Build a chatgpt bot that will analyze the text given and try to locate any false info

In [10]:
chat_completion  = client.chat.completions.create(
    model = "gpt-4", # Quote
    messages = [{"role" : "system", "content" : "You're a summary assistant at wekepedia, i will pass you an article and please tell me if any of the information is false"},
              {"role" : "user", "content" : content[:8192]}] 
)

In [11]:
chat_completion.choices[0].message.content

"The information provided is accurate and provides a well-rounded insight into the field of renewable energy, its sources, benefits, uses, associated jobs, and future growth. The fact that renewable resources - including sunlight, wind, water movement, and geothermal heat - are naturally renewable and contrast fossil fuels which are rapidly depleting, is correctly stated. The increase in the global electricity supply from renewable sources from 20% in 2011 to 28% in 2021, the job opportunities in the sector, as well as the obstacles that hinder the growth of renewables are also well represented. The detailed breakdown of the contribution of different renewable sources like solar and wind power, the drop in hydropower's share, the growth of biomass and geothermal energy, is confirmed. Information regarding the rise in the use and efficiency of renewable energy, and its increasing affordability is correctly indicated. The emphasis on achieving net zero emissions by 2050 from the Internat

In [12]:
def chatgpt_error_correction(text):
    chat_completion  = client.chat.completions.create(
    model = "gpt-4",
    messages = [{"role" : "system", "content" : "I will be giving you an article let me know if anything is potentially false.  Go with afine tooth comb and have low sensitivity and locating potential errors. If there is errors please number them and be very concise when quoting them. If there is no potential errors found then just return 'NONE'"},
              {"role" : "user", "content" : text[:8192]}] # List
)
    return print(chat_completion.choices[0].message.content)

# 4.) Make a for loop and check a few wikipedia pages and return a report of any potentially false info via wikipedia

In [13]:
page_titles = ["Artificial Intelligence", "UCLA"]
for page_title in page_titles:
    print("__________________" + page_title)
    try:
        text =  get_wikipedia_content(page_title)
        print(chatgpt_error_correction(text[:8192]))
    except:
        print("ERROR")

__________________Artificial Intelligence
NONE
None
__________________UCLA
1. "It received 174,914 undergraduate applications for Fall 2022, including transfers, making it the most applied-to university in the United States." This statement could be incorrect as it depends on the source of data and the year the article was written. It would require verification from a reliable source for the year mentioned.

2. "Six of the schools offer undergraduate degree programs: Arts and Architecture, Engineering and Applied Science, Music, Nursing, Public Affairs, and Theater, Film and Television." UCLA has more undergraduate schools it includes the School of Theater, Film, and Television as an undergraduate school, not Public Affairs. There is an undergraduate program in Public Affairs, but it is housed within the Luskin School of Public Affairs, which is primarily a graduate school.

3. "On June 1, 2016, two men were killed in a murder-suicide at an engineering building in the university." This